In [11]:
from langchain_community.chat_models import ChatZhipuAI

ZHIPUAI_API_KEY = "0f2f6aca4fb384e90505b066a1ddf204.hM6JYEdsgvTYLxWz"
llm = ChatZhipuAI(
    temperature=0.1,
    api_key=ZHIPUAI_API_KEY,
    model_name="glm-4",
)

from langchain_community.utilities import SQLDatabase
db = SQLDatabase.from_uri("sqlite:///G:/Python Project/llm+sql/ali_langchain.db")

def get_schema(_):
    return db.get_table_info()
    
def run_query(query):
    return db.run(query)

result = run_query("SELECT COUNT(*) FROM alidata")
print(db.get_table_info())


CREATE TABLE alidata (
	"index" INTEGER, 
	user INTEGER, 
	brand INTEGER, 
	behavr INTEGER, 
	date TEXT
)

/*
3 rows from alidata table:
index	user	brand	behavr	date
0	10944750	13451	0	06/04
1	10944750	13451	2	06/04
2	10944750	13451	2	06/04
*/


In [8]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableLambda

template_sql = (
    "请通过写sql代码来回答对应问题，并且需要基于如下数据库信息：{info}\n"
    "需要回答的问题是：{question}\n"
    "注意仅需要通过sql代码回答，不需要文字\n"
    "代码形式如下：\n"
    "```sql\n"
    "...\n"
    "```"
)
t_sql=PromptTemplate.from_template(template_sql)

def get_sql(x):
    return x.split("```sql")[1].split("```")[0]

chain_sql=({"info":get_schema,"question":RunnablePassthrough()}
           |t_sql
           |llm
           |StrOutputParser()
           |RunnableLambda(get_sql))

template_sql0=(
    "请通过综合如下的数据库信息，问题，sql代码，sql代码的执行结果给出问题的自然语言回答：\n" 
    "数据库信息{info} \n"
    "需要回答问题是：{question} \n" 
    "sql代码：{query} \n" 
    "sql代码执行结果: {res}"
)
t_sql0=PromptTemplate.from_template(template_sql0)

chain_sql0=({"info":get_schema,"question":RunnablePassthrough(),"query":chain_sql}
            |RunnablePassthrough.assign(res=lambda x: run_query(x["query"]))
            |t_sql0
            |llm
            |StrOutputParser())

In [15]:
chain_sql.invoke("date最高的是哪一个客户？，是多少？")

'\nSELECT user, MAX(date) AS latest_date\nFROM alidata\nGROUP BY user\nORDER BY latest_date DESC\nLIMIT 1;\n'

In [16]:
chain_sql0.invoke("date最高的是哪一个客户？是多少？")

"根据提供的数据库信息以及执行的SQL代码及结果，可以得出以下自然语言回答：\n\n在alidata表中，日期（date）最高的客户是用户编号为12386500的客户，其最高日期记录为'08/15'。"